# Veracity Project: Distributed Ledgers and Prolog
## Hayden McAlister - University of Otago 2022

### Running this demonstration
This demonstration is (currently) hosted on [Catalyst Cloud](https://catalystcloud.nz/). However, the IP address associated with this project is floating and could change regularly. If desired, this could be assigned some fixed domain name so this demonstration could be accessed anywhere, anytime.

Otherwise, this demonstration is accessible from GitHub as a docker-compose system. [This repository](TODO) describes how to download, run, and interact with this demonstration locally. 

---

## The infrastructure

In this research project we investigated how distributed ledger technology could be used in a veracity context to verify claims and reach consensus on the state of a system. Other research was done concurrently into using Prolog to describe the state of a system and look at provenance in an organ donation scenario. I built the infrastructure to integrate this Prolog into a jupyter notebook (for accessibility and easy interaction) and expose the blockchain for use.

We agreed that for a proof of concept a small scale, locally hosted blockchain would be most suitable. There is no need to put our tests on a "real" ledger at this stage, but if this project moves forward in this direction it may be useful to put information on a public ledger so it can be verified.

My first task was to find a suitable blockchain technology that could be locally hosted. I found [Hyperledger](https://www.hyperledger.org/) to be very promising, as the opensource projects could be locally hosted and seemed very flexible to cover many situations. I looked in to using Hyperledger [Indy](https://www.hyperledger.org/use/hyperledger-indy), [Aries](https://www.hyperledger.org/use/aries), [Besu](https://www.hyperledger.org/use/besu), and [Ursa](https://www.hyperledger.org/use/ursa) but determined that [Hyperledger Iroha](https://www.hyperledger.org/use/iroha) would be most suitable for my needs. This is mainly due to Iroha having the best documentation of the projects I looked at, and was the easiest to start developing with quickly. However, other projects could absolutely be used and in future local hosting may not even be needed.

I spent several weeks setting up infrastructure using Iroha and Docker. I managed to get four Iroha nodes running and comunicating in separate containers, which is significant as four nodes is the minimum neccisary to detect and correct for a single faulty node (crash fault or byzantine fault). While this demonstration does not look any further into faulty nodes, this infrastructure would support probing in this direction.

We can see these containers running. If running this demonstration locally, you can run 

`docker container ls`

and spot the containers labeled `iroha1` through `iroha4`. These containers are hosting the blockchain nodes and are constantly communicating to see if new transactions have arrived. These nodes are backed by the containers `some-postgres1` through `some-postgres4`. I am sure with enough convincing the postgres containers could be consolidated into a single container with several logins (one for each Iroha node), however this infrastructure was the easiest to develop quickly. 

We can also see what the current state of the blockchain is by running the following command. We will discuss this command later on, but for now just know it will query `iroha1` and get all blocks on the chain, storing them in `logs/blocks.log`.

In [4]:
%python
from IrohaUtils import *
log_all_blocks("blocks.log")

We can see in this file that the blocks are represented in JSON format, and are mostly incomprehensible jargon. A lot of this can be understood by trawling through [the documentation](https://iroha.readthedocs.io/en/main/), however this is time consuming and very dense (I said the documentation was the best of the projects above, I did not claim the documentation was clear). For this reason, I have developed some tools we will discuss soon that make developing and interacting with the Iroha network much easier. 

You may also notice other containers running due to this network. There is the `swipl-notebook` container which, unsurprisingly, runs the jupyter notebook server you are interacting with now! There is also a redis container which we will discuss later on.

Because the goal of this project was to eventually host some Prolog work, we had to get Prolog running in a jupyter notebook. Thankfully this task was done for us (and can be found [here](https://github.com/veracitylab/jupyter-swi-prolog)), and with some scripting the jupyter notebook can run Prolog code! The kernel essentially runs Prolog line by line through a python interface with SWIPL, then dumps the factbase into a file for consultation later. These files are referenced by Prolog when querying the factbase, so the files store the state of the world. This will be useful later.

---

### Kernel modifications

The kernel, while fantastic, needed extending to work with the blockchain network. I added some extra functionality which made development of Prolog with the blockchain a little easier.

First, the kernel checks environment variables to determine state. This means we can easily turn on or off functionality as needed. The supported variables are:

- BLOCKCHAIN: 0 or 1. Boolean check to interact with the blockchain or not. If 1 then after executing a Prolog cell the kernel will hash the fact file and store this hash on the blockchain. This provides proof that a cell was run and was added to a fact base.
- REDIS: 0 or 1. Boolean check to interact with the redis container or not. If 1 then after execution of a Prolog cell the kernel will map the hash of the fact file with the fact file text itself. This means that if a hash is known (say, if we found it on the blockchain) then we can retrive the file that generated it. Currently, these files are stored unencrypted
- TIMESTAMPING: 0 or 1. Boolean check to add a timestamp to the fact files before hashing. If 0, no timestamp is added. This means that if a cell is run multiple times then only the first fact file can be stored. In particular, this means that if a cell undergoes a change from state A to B to A, then the blockchain will be unable to store the change back to state A. 
- LOGGING_LEVEL: Integer from 0 to 50. Determines the level of logging. The most useful levels are 10 (DEBUG) and 20 (INFO)

It would be nice to set these environment variables from within a notebook, so I added functionality to do this using "magic command" syntax. Starting a cell with `%ENV` will allow you to set environment variables as needed.

In [7]:
%ENV
BLOCKCHAIN=0
REDIS=1
TIMESTAMPING=1
LOGGING_LEVEL=20

SET ENV BLOCKCHAIN=0
SET ENV REDIS=1
SET ENV TIMESTAMPING=1
SET ENV LOGGING_LEVEL=20

Other magic commands are supported. For example, to change the fact file when running Prolog, we can use the `%file` command: (note that fact files are stored under the `consulted_files` directory.

In [8]:
%file: foo.pl

man(socrates).
mortal(X) :- man(X).

Redis storing b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash

We can use `%python` to run python code in a Prolog notebook:

In [6]:
%python
x=1
y=2
print(f"{x+y=}")

x+y=3

And we can consult files from the REDIS database using `%consult`:

In [9]:
%consult
b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash

b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash
% 1643674741031871443
man(socrates).
mortal(X) :- man(X).
--------------------------------------------------------------------------------
Successfully consulted foo.pl
--------------------------------------------------------------------------------


Using these extra kernel functions we can easily develop some example scenarios to demonstrate how this technology could be used in a veracity context

---

### Iroha Utilities

The final part of my project to discuss before we are able to move in to demonstrations is the Iroha utilities I have created. As discussed above, the Iroha network is moderately well documented, and creating applications with this documentation takes a long time. For this reason I have created the IrohaUtils package which provides many functions to quickly interact with the Iroha network.